In [1]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
from wagtail.admin.auth import get_user_model
User = get_user_model()

In [2]:
USERS_YAML = """
-
    user:
        username: admin
        email: admin@email.io
        first_name: Amin
        last_name: L'admin
        password: admin
    role: admin
-
    user:
        username: pduval
        email: pduval@email.io
        first_name: Pierre
        last_name: Duval
        password: pduval
    role: Editors
-
    user:
        username: jprevert
        email: jprevert@email.io
        first_name: Jacques
        last_name: Prévert
        password: jprevert
    role: Editors
-
    user:
        username: rdurand
        email: rdurand@email.io
        first_name: Robert
        last_name: Durand
        password: pduval
    role: Moderators
-
    user:
        username: jconrad
        email: jconrad@email.io
        first_name: Joseph
        last_name: Conrad
        password: jconrad
    role: Moderators
-
    user:
        username: gorwell
        email: gorwelld@email.io
        first_name: George
        last_name: Orwell
        password: gorwell
    role: Customers
"""


In [3]:
import yaml
usersdata = yaml.safe_load(USERS_YAML)

In [4]:
# from wagtail.
from django.contrib.auth.models import Group

In [5]:
group = Group(name='Customers')

In [6]:
group.save()

In [7]:
# help(group)

In [8]:
for data in usersdata:
    role = data['role']
    if role == 'admin':
        print('admin')
    else:
        try:
            group = Group.objects.filter(name=role)[0]
            print('role: group:' + str(group))
            # print('type:' + str(type(group)))
        except IndexError:
            print('group {:s} not found'.format(role))
            print('group creating {:s}...'.format(role))
            group = Group(name=role)
            group.save()
            


admin
role: group:Editors
role: group:Editors
role: group:Moderators
role: group:Moderators
role: group:Customers


In [9]:
[u['user']['username'] for u in usersdata]

['admin', 'pduval', 'jprevert', 'rdurand', 'jconrad', 'gorwell']

In [10]:
def delete_users(usersdata):
    usernames = [u['user']['username'] for u in usersdata]
    for username in usernames:
        q = User.objects.filter(username=username)
        if len(q) > 0:
            print('{:s} deleting...'.format(username))
            q[0].delete()
        else:
            print('{:s} not found '.format(username))

In [11]:
def list_users():
    print(get_user_model().objects.all())

In [12]:
def populate_users(usersdata, delete=False):
    userclass = get_user_model()

    for data in usersdata:
        role = data['role']
        userdata = data['user']
        username = userdata['username']
        q = userclass.objects.filter(username=username)
        if len(q) > 0:
            # remove any user with that name
            if delete:
                try:
                    q[0].delete()
                except IndexError:
                    pass

        if role == 'admin':
            user = userclass.objects.create_superuser(**userdata)
            user.save()
        else:
            group = None
            try:
                group = Group.objects.filter(name=role)[0]
            except IndexError:
                print('group {:s} not found creating...'.format(role))
                group = Group.objects.create(name=role)

            user = userclass.objects.create_user(**userdata)
            if group:
                user.groups.add(group)
            user.save()


In [13]:
list_users()
print(Group.objects.all())

<QuerySet []>
<QuerySet [<Group: Moderators>, <Group: Editors>, <Group: Customers>]>


In [14]:
User.objects.all()

<QuerySet []>

In [15]:
# delete_users(usersdata)

In [16]:
populate_users(usersdata)

In [17]:
list_users()

<QuerySet [<User: admin>, <User: pduval>, <User: jprevert>, <User: rdurand>, <User: jconrad>, <User: gorwell>]>
